In [ ]:
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt
from utils import calcBg
import os

'''
process DFF of raw images (slow!), save npy in outputs/sample-screening-traces
all from same imaging plate (rig 2, 177 FPS)

'''
def calcDFF(name, _tif_dir_,_n_AP_):
    

    img = tiff.imread(_tif_dir_)
    img = img[::2,:,:] # downsample by 2
    n_frames = img.shape[0]
    
    img_max = np.amax(img, axis=0) # max projection
    img_max_flat = img_max.flatten()
    img_max_flat_sorted = np.flip(np.sort(img_max_flat))
    nel = len(img_max_flat)
    min_bright_val = img_max_flat_sorted[int(np.round(percent_bright / 100 * nel))]

    bg = [calcBg(b, percent_bg) for b in img] # bg: [nframesx1]

    mask_idx = img_max_flat > min_bright_val
    img_masked = np.reshape(img, (n_frames, nel))
    f = img_masked.mean(axis=1)
    f0 = f[10:20].mean()
    dff = (f-f0) / (f0-bg)
    
    np.save(os.path.join('outputs/sample-screening-traces/{}AP'.format(_n_AP_), name + '.npy'), dff)


nAP = 40
main_dir = r'Z:/GETIScreenData/RawImageData/20201104_iGABASnFR_rig2/P16a-20201019_iGABASnFR'

#1AP
names_and_dirs_1AP = {'4409' : os.path.join(main_dir,'P16a-20201019_96Well70-F10_514dot4409_X-88795_Y-50728_Z8625.08_1AP_sweep175.tif'),
                 'F102G': os.path.join(main_dir,'P16a-20201019_96Well87-H03_514dot1669_X-25795_Y-68728_Z8550.32_1AP_sweep214.tif'),
                 'iGABA': os.path.join(main_dir,'P16a-20201019_96Well66-F06_514dot1_X-52795_Y-50728_Z8615.09_1AP_sweep163.tif'),
                 '4445': os.path.join(main_dir,'P16a-20201019_96Well35-C11_514dot4445_X-97795_Y-23728_Z8665.24_1AP_sweep88.tif')
                 }


# 40AP
names_and_dirs_40AP = {'4409' : os.path.join(main_dir,'P16a-20201019_96Well70-F10_514dot4409_X-88795_Y-50728_Z8625.08_40AP_sweep177.tif'),
                 'F102G': os.path.join(main_dir,'P16a-20201019_96Well87-H03_514dot1669_X-25795_Y-68728_Z8550.32_40AP_sweep216.tif'),
                 'iGABA': os.path.join(main_dir,'P16a-20201019_96Well66-F06_514dot1_X-52795_Y-50728_Z8615.09_40AP_sweep165.tif'),
                 '4445': os.path.join(main_dir,'P16a-20201019_96Well35-C11_514dot4445_X-97795_Y-23728_Z8665.24_40AP_sweep90.tif')
                 }

# img_dir = r'Z:/GETIScreenData/RawImageData/20201104_iGABASnFR_rig2/P16a-20201019_iGABASnFR/P16a-20201019_96Well70-F10_514dot4409_X-88795_Y-50728_Z8625.08_40AP_sweep177.tif'
percent_bg = 5
percent_bright = 5 # top % pixels to include in mask

names_and_dirs = names_and_dirs_40AP if (nAP == 40) else names_and_dirs_1AP
for n in names_and_dirs.keys():
    calcDFF(n, names_and_dirs[n], nAP)

In [ ]:
'''
plot sample screening traces
load npy in outputs/sample-screening-traces/{1,40}AP and plot
'''
import matplotlib.pyplot as plt
import numpy as np

def exp_fit(x, a, b, c):
    return a * np.exp(-b*x) + c

def debleach(f_trace, _t):
    '''
    simple exp fit
    f_trace: [1xN] F trace
    t: [1xN] entire time vector
    '''
    fit_idx = np.concatenate((np.arange(50), np.arange(-50,-1)))
    x_fit = _t[fit_idx]
    y_fit = f_trace[fit_idx]
    popt, pcov = curve_fit(exp_fit, x_fit, y_fit)
    
    return f_trace - exp_fit(t, *popt)

nAPs = ['1', '40']
constructs_to_plot = {'iGABA':'black', 
                      'F102G':'grey', 
                      '4409': 'red', 
                      '4445': 'royalblue'}
for AP in nAPs:
    # plot all DFF
    

    s_rate = 177 / 2 # 177 Hz, downsampled x2

    f = plt.figure()
    for c in constructs_to_plot.keys():
        c_dff = np.load('outputs/sample-screening-traces/{}AP/'.format(AP) + c + '.npy')

        # remove fast bleach part
        c_dff = c_dff[10:]
        t = np.arange(0,len(c_dff))/s_rate
    
        # debleach unless it's 4409 or 4445 at 40 AP (does not go back to baseline)
        dff_debleach = c_dff if (AP=='40' and (c == '4409' or c == '4445')) else debleach(c_dff,t)
        plt.plot(t, dff_debleach, color=constructs_to_plot[c])

    plt.legend(constructs_to_plot)
    plt.ylabel('$\Delta$F/F', fontsize=14)
    plt.xlabel('t (s)', fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    f.savefig('figs/sample-screening-traces_{}AP.pdf'.format(AP))